# Análise dos produtos comercializados

## Criar directoria de apoio 

In [1]:
from pathlib import Path

Path('support').mkdir(parents=True, exist_ok=True)
Path('reports').mkdir(parents=True, exist_ok=True)

## Criar ficheiro de configuração se não existir

O ficheiro de configuração serve para armazenar
a APIKEY que dá acesso a mais de 500 linhas de
resultados.


In [2]:
import os

fname = 'config.ini'
content = """
# Ficheiro de configuração
[comtrade]
# Add API Key. DO NOT SHARE
key = APIKEYHERE
"""
if not os.path.isfile(fname):
  print("Criando ficheiro de configuração")
  with open(fname,'w') as f:
    f.write(content)


## Obter uma chave de acesso à API

Para aceder à UN Comtrade via API sem limites é necessário uma chave de acesso,
de outro modo os resultados são limitados a 500 linhas.

Para obter a chave de acsso:
* Registo em https://comtradedeveloper.un.org/
* Ir para _Products_ 
* Selecionar "Premium Individual APIs" (https://comtradedeveloper.un.org/product#product=dataapis)
* Escolher _Subscribe to "comtrade - v1"_  
* Esperar pelo email com a chave da API key (demora alguns dias)
* Copiar a chave para o local indicado no ficheiro `config.ini` antes
  de executar o resto do notebook.


In [3]:
import configparser
import comtrade

if os.path.isfile(fname):

    config = configparser.ConfigParser()
    config.read('config.ini')
    APIKEY = config['comtrade']['key']

comtrade.init(APIKEY)

## Parâmetros gerais que não mudam



In [4]:
m49_angola = 24
m49_brazil = 76
m49_cabo_verde = 132
m49_china = 156
m49_hong_kong = 344
m49_macau = 446
m49_guine_equatorial = 226
m49_guine_bissau = 624
m49_mozambique = 508
m49_portugal = 620
m49_stome_principe = 678
m49_timor = 626

# make list of Portuguese Speaking Countries
m49_plp = [m49_angola,m49_brazil,m49_cabo_verde,m49_guine_bissau,
            m49_guine_equatorial,m49_mozambique,m49_portugal,
            m49_stome_principe,m49_timor]
m49_plp_list = ",".join(map(str,m49_plp))




## China, categorias de produtos mais importantes nas trocas com os PLP

Obtem os totais agregados com nível 2 de código HS e lista os primeiros.

Para significado detalhado de cada código ver: https://www.wcoomd.org/en/topics/nomenclature/instrument-and-tools/hs-nomenclature-2022-edition/hs-nomenclature-2022-edition.aspx


### TODO

* [_] Permitir escolher os reporter e partners
* [_] compatibilizar com o formato de report da AICEP https://myaicep.portugalexporta.com/mercados-internacionais/cn/china?setorProduto=-1
* [_] há problemas de duplicação de linhas nas listagem de detalhes de commodities, provavlmente por custom code, ou mot code.

In [5]:
import comtrade
import pandas as pd


rank_filter = 5  # número de importações mais relevantes
years = "2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021"
flowCode = 'M,X'
pco_cols = ['reporterDesc','partnerDesc','refYear','rank','cmdCode','cmdDesc',
            'flowCode','primaryValue']
df = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flowCode,
                     cmdCode="AG2",
                     reporterCode=m49_china,
                     partnerCode=m49_plp_list,
                     period=years,
                     timeout=120
                     )

pco = df.sort_values(['partnerDesc','refYear','primaryValue'], ascending=[True,True,False])
pco['rank'] = pco.groupby(['partnerDesc','refYear','flowCode'])["primaryValue"].rank(method="dense", ascending=False)
# convert rank column to int
pco['rank'] = pco['rank'].astype(int)

pco_top5 = pco[pco['rank'] <= rank_filter]
cmdCodes_top5 = pco_top5['cmdCode'].unique()
print(f"Categorias de produtos envolvidas: {cmdCodes_top5}")
# save to Excel
pco_top5_sorted = pco_top5[pco_cols].set_index(['reporterDesc','partnerDesc','refYear','flowCode','rank']).sort_index()

# Prepare file name for Excel output
filename_note=f"{years.replace(',','_')}_{flowCode.replace(',','_')}"  # change to append to filename
excel_file_name = f"./reports/china_plp_top5_{filename_note}.xlsx"
excel_file = pd.ExcelWriter(excel_file_name)
# Save data
pco_top5_sorted.to_excel(excel_file, sheet_name="comtrade", startrow=2)
excel_file.close()
# Write title in first row TBD
excel_file_title = f"Trocas Comerciais China-PLP, top {rank_filter} produtos (M=Importações, X=Exportações), {years}, valores USD"
print(excel_file_title)
print("Guardado em:",excel_file_name)

# show
pd.options.display.max_colwidth=100
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 100
pco_top5_sorted.head(100)

Categorias de produtos envolvidas: ['27' '64' '07' '85' '62' '34' '44' '73' '39' '63' '03' '25' '88' '52'
 '16' '83' '87' '84' '71' '20' '90' '40' '72' '74' '76' '94' '68' '22'
 '01' '26' '12' '47' '29' '24' '15' '54' '41' '17' '02' '89' '61' '82'
 '95' '55' '10' '70' '60' '09' '69' '99' '92' '49' '38' '30' '58' '48'
 '65' '97' '33' '56' '08' '04' '21' '96' '28' '31' '86' '42' '45' '91'
 '32' '06' '14' '78']
Trocas Comerciais China-PLP, top 5 produtos (M=Importações, X=Exportações), 2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021, valores USD
Guardado em: ./reports/china_plp_top5_2000_2001_2002_2003_2004_2005_2006_2007_2008_2009_2010_2011_2012_2013_2014_2015_2016_2017_2018_2019_2020_2021_M_X.xlsx


cmdCode  \
reporterDesc partnerDesc refYear flowCode rank           
China        Angola      2000    M        1         27   
                                          2         44   
                                          3         73   
                                          4         39   
                                          5         85   
                                 X        1         64   
                                          2         07   
                                          3         85   
                                          4         62   
                                          5         34   
                         2001    M        1         27   
                                          2         44   
                                          3         03   
                                          4         25   
                                          5         88   
                                 X        1         64   
                                          2         62   
                                          3         85   
                                          4         34   
                                          5         63   
                         2002    M        1         27   
                                          2         03   
                                          3         44   
                                          4         16   
                                          5         83   
                                 X        1         64   
                                          2         07   
                                          3         85   
                                          4         62   
                                          5         52   
                         2003    M        1         27   
                                          2         25   
                                          3         71   
                                          4         39   
                                          5         20   
                                 X        1         85   
                                          2         64   
                                          3         87   
                                          4         52   
                                          5         84   
                         2004    M        1         27   
                                          2         71   
                                          3         25   
                                          4         90   
                                          5         40   
                                 X        1         85   
                                          2         87   
                                          3         64   
                                          4         84   
                                          5         39   
                         2005    M        1         27   
                                          2         71   
                                          3         73   
                                          4         44   
                                          5         03   
                                 X        1         87   
                                          2         85   
                                          3         84   
                                          4         64   
                                          5         73   
                         2006    M        1         27   
                                          2         71   
                                          3         25   
                                          4         72   
                                          5         74   
                                 X        1         85   
                                          2         87   
                                          3  

## Detalhe do tipo de produtos comercializados entre os PLP e a China

Em dois passos: 
* obtém as categorias de nível 2 mais importantes de cada país
* pesquisa todos as subcategorias de cada um das categorias

Para significado detalhado de cada código ver: https://www.wcoomd.org/en/topics/nomenclature/instrument-and-tools/hs-nomenclature-2022-edition/hs-nomenclature-2022-edition.aspx

In [6]:

import ipywidgets as widgets
from IPython.display import display

rank_filter = 5  # número de importações mais relevantes
years = "2021"
partnerCode = m49_angola # 
flowCode="M"

# select year with widget from list range(2000,2022)

years = widgets.SelectMultiple(
    options=range(2000,2022),
    value=[2021],
    description='Ano:',
    disabled=False,
)

# select country widget from list Angola, Brazil, Cabo Verde, Guiné Bissau, Equatorial Guinea, Mozambique, Portugal, São Tomé e Príncipe, Timor-Leste
reporterCodeWidget = widgets.Dropdown(
    options=[("China", m49_china),("Angola", m49_angola), ("Brazil", m49_brazil), ("Cabo Verde", m49_cabo_verde), ("Guiné Bissau", m49_guine_bissau), ("Guiné Equatorial", m49_guine_equatorial), ("Mozambique", m49_mozambique), ("Portugal", m49_portugal), ("São Tomé e Príncipe", m49_stome_principe), ("Timor-Leste", m49_timor)],
    # value=[("Angola",m49_angola)],
    description='Fonte (reporter):',
    disabled=False,
)

# select country widget from list Angola, Brazil, Cabo Verde, Guiné Bissau, Equatorial Guinea, Mozambique, Portugal, São Tomé e Príncipe, Timor-Leste
partnerCodeWidget = widgets.Dropdown(
    options=[("Angola", m49_angola), ("Brazil", m49_brazil), ("Cabo Verde", m49_cabo_verde), ("Guiné Bissau", m49_guine_bissau), ("Guiné Equatorial", m49_guine_equatorial), ("Mozambique", m49_mozambique), ("Portugal", m49_portugal), ("São Tomé e Príncipe", m49_stome_principe), ("Timor-Leste", m49_timor),("China", m49_china),],
    # value=[("Angola",m49_angola)],
    description='Parceiro:',
    disabled=False,
)

# select flowCode widget from list M=Importações, X=Exportações
flowCodeWidget = widgets.Dropdown(
    options=[("Importações", "M"), ("Exportações", "X")],
    value="M",
    description='Fluxo:',
    disabled=False, 
)



### Escolher ano, parceiro e fluxo (import/export) para análise 

Pode escolher-se mais que um ano.

In [13]:

display(years)
display(reporterCodeWidget)
display(partnerCodeWidget)
display(flowCodeWidget)


SelectMultiple(description='Ano:', index=(21,), options=(2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008,…

Dropdown(description='Fonte (reporter):', index=7, options=(('China', 156), ('Angola', 24), ('Brazil', 76), ('…

Dropdown(description='Parceiro:', index=9, options=(('Angola', 24), ('Brazil', 76), ('Cabo Verde', 132), ('Gui…

Dropdown(description='Fluxo:', options=(('Importações', 'M'), ('Exportações', 'X')), value='M')

Depois de escolher ano, país e fluxo executar a célula seguinte.

Para significado detalhado de cada código ver: https://www.wcoomd.org/en/topics/nomenclature/instrument-and-tools/hs-nomenclature-2022-edition/hs-nomenclature-2022-edition.aspx

In [14]:
import comtrade
from comtrade import COUNTRY_CODES, HS_CODES, HS_CODES_DF, HS_CODES_L2_DF


yearsList = ",".join(list(map(str,years.value)))
partnerCode = partnerCodeWidget.value
reporterCode = reporterCodeWidget.value


# decode country
partner_name = comtrade.COUNTRY_CODES.get(partnerCode)
reporter_name = comtrade.COUNTRY_CODES.get(reporterCode)

flow = flowCodeWidget.value
print(reporter_name,yearsList,flow,partner_name,)

pco_cols = ['reporterDesc','partnerDesc','refYear','rank','cmdDesc',
            'flowCode','primaryValueFormated']

pco_cols_detail = ['reporterDesc','partnerDesc','refYear','cmdCodeAG2','cmdCode','cmdDesc',
            'flowCode','primaryValue', 'isAggregate']

# first we collect the top commodity level 2 codes            
df = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flow,
                     cmdCode="AG2",
                     reporterCode=reporterCode,
                     partnerCode=partnerCode,
                     period=yearsList,
                     timeout=30,
                     echo_url=True
                     )

pco = df.sort_values(['partnerDesc','refYear','primaryValue'], ascending=[True,True,False])
pco['rank'] = pco.groupby(['partnerDesc','refYear'])["primaryValue"].rank(method="dense", ascending=False)
pco_top5 = pco[pco['rank'] <= rank_filter]
# get the countries
countries = pco_top5.partnerDesc.unique()
country_cmd_top5_codes = dict()

# get the detailed commodity codes for the top of each country
for country in countries:
    l2_codes = pco_top5[pco_top5.partnerDesc == country]['cmdCode'].unique()
    print(country,l2_codes)
    hs_details = []
    for l2_code in l2_codes:
        l2_sub_codes = list(HS_CODES_DF[HS_CODES_DF.hscode.str.startswith(l2_code)]['hscode'])
        hs_details = hs_details + l2_sub_codes
    # print(hs_details)
    country_cmd_top5_codes[country] = hs_details.copy()

# now we fetch the detail
countryDesc = COUNTRY_CODES[partnerCode]
country_cmd_codes = ",".join(country_cmd_top5_codes[countryDesc])
print(country_cmd_codes)

df = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flow,
                     #cmdCode=country_cmd_codes, # sometimes the detail commodity codes generates errors
                     cmdCode=None, # This gives all the commodities for the country and year 
                     reporterCode=reporterCode,
                     partnerCode=partnerCode,
                     partner2Code=0,
                     period=yearsList,
                     echo_url=True,
                     timeout=60
                     )
df['cmdCodeAG2'] = df.cmdCode.str[0:2]
# filter the detail commodity codes by the top 5
df = df[df.cmdCode.isin(country_cmd_top5_codes[countryDesc])]

df['subtotalAG2'] = df.groupby(['partnerCode','refYear','flowCode','cmdCodeAG2'])["primaryValue"].transform('sum')

df[pco_cols_detail].sort_values('primaryValue',ascending=False).to_excel(f"./reports/product_detail_{reporter_name}_{yearsList}_{flowCode}_{partner_name}.xlsx")

pd.options.display.max_rows = 500
df.sort_values(['partnerDesc','refYear','flowCode','subtotalAG2','cmdCodeAG2','primaryValue'],ascending=[True,True,True,False,True,False])[pco_cols_detail].head(500)

Portugal 2021 M China
https://comtradeapi.un.org/data/v1/get//C/A/HS?reporterCode=620&period=2021&partnerCode=156&partner2Code=0&cmdCode=AG2&flowCode=M&customsCode=C00&subscription-key=HIDDEN
China ['85' '84']
85,8501,850110,850120,850131,850132,850133,850134,850140,850151,850152,850153,850161,850162,850163,850164,850171,850172,850180,8502,850211,850212,850213,850220,850231,850239,850240,8503,850300,8504,850410,850421,850422,850423,850431,850432,850433,850434,850440,850450,850490,8505,850511,850519,850520,850590,8506,850610,850630,850640,850650,850660,850680,850690,8507,850710,850720,850730,850750,850760,850780,850790,8508,850811,850819,850860,850870,8509,850940,850980,850990,8510,851010,851020,851030,851090,8511,851110,851120,851130,851140,851150,851180,851190,8512,851210,851220,851230,851240,851290,8513,851310,851390,8514,851411,851419,851420,851431,851432,851439,851440,851490,8515,851511,851519,851521,851529,851531,851539,851580,851590,8516,851610,851621,851629,851631,851632,851633,

,reporterDesc,partnerDesc,refYear,cmdCodeAG2,cmdCode,cmdDesc,flowCode,primaryValue,isAggregate
15360,Portugal,China,2021,85,85,Electrical machinery and equipment and parts thereof; sound recorders and reproducers; televisio...,M,"1,458,244,678.71",True
15365,Portugal,China,2021,85,85,Electrical machinery and equipment and parts thereof; sound recorders and reproducers; televisio...,M,"1,023,243,360.48",True
11052,Portugal,China,2021,85,8541,"Semiconductor devices (e.g. diodes, transistors, semiconductor based transducers); including pho...",M,"429,686,488.96",True
10999,Portugal,China,2021,85,8541,"Semiconductor devices (e.g. diodes, transistors, semiconductor based transducers); including pho...",M,"424,321,043.13",True
15362,Portugal,China,2021,85,85,Electrical machinery and equipment and parts thereof; sound recorders and reproducers; televisio...,M,"281,755,281.35",True
11729,Portugal,China,2021,85,8529,Transmission apparatus; parts suitable for use solely or principally with the apparatus of headi...,M,"158,176,390.40",True
14422,Portugal,China,2021,85,852990,"Reception and transmission apparatus; for use with the apparatus of heading no. 8524 to 8528, ex...",M,"151,211,270.56",True
15364,Portugal,China,2021,85,85,Electrical machinery and equipment and parts thereof; sound recorders and reproducers; televisio...,M,"142,326,599.86",True
12542,Portugal,China,2021,85,8517,"Telephone sets, including smartphones and other telephones for cellular/wireless networks; other...",M,"124,910,187.00",True
9780,Portugal,China,2021,85,8534,Circuits; printed,M,"121,205,591.24",True
